This is a tutorial for the DFODE-kit package.

In [ ]:
import os

import numpy as np
import matplotlib.pyplot as plt

from dfode_kit.df_interface import (
    OneDFreelyPropagatingFlameConfig,
    setup_one_d_flame_case,
    df_to_h5,
)
from dfode_kit.data_operations import (
    touch_h5, 
    get_TPY_from_h5, 
    random_perturb,
    label_npy,
    integrate_h5,
    calculate_error,
)
from dfode_kit.dfode_core.model.mlp import MLP
from dfode_kit.utils import BCT

DFODE_ROOT = os.environ['DFODE_ROOT']

print(DFODE_ROOT)

eq_ratios = [0.8, 1.0, 1.2]

### A brief introduction to the DFODE method

#### Low-dimensional manifold sampling

A key challenge in preparing training data is achieving sufficient coverage of the relevant thermochemical composition space, which is often prohibitively high-dimensional when detailed chemistry involves tens to hundreds of species. 

To address this, DFODE-kit adopts a low-dimensional
manifold sampling strategy, where thermochemical states are extracted from canonical flame configurations that retain the essential topology of high-dimensional turbulent flames. This approach ensures both computational efficiency and physical representativeness of the training datasets.

In this tutorial, we will demonstrate how to use DFODE-kit to sample a low-dimensional manifold of thermochemical states from a one-dimensional laminar freely propagating flame simulated with DeepFlame. The following code block could also be found in `dfode_kit_init.ipynb` files within the case templates provides in the `canonical_cases` directory. It is used to initialize the simulation and update the dictionary files for the simulation.

Note:
For this demonstration, a number of 10 for output steps is adopted per case. This parameter can be adjusted upward in actual implementations to enhance statistical reliability.

In [ ]:
from pathlib import Path
import shutil
standard_case = 'standard_case_for_sampling'

num_output_steps = 100

standard_case_path = f"{DFODE_ROOT}/tutorials/twoD_HIT_flame/1_sample_train/{standard_case}"

for eq_ratio in eq_ratios:
    new_case_name = f"eq_ratio_{eq_ratio}"
    new_case_path = f"{DFODE_ROOT}/tutorials/twoD_HIT_flame/1_sample_train/{new_case_name}"

    if os.path.exists(standard_case_path):
        shutil.copytree(standard_case_path, new_case_path)
        print(f"copy {standard_case} to {new_case_name}")

        original_dir = os.getcwd()
        os.chdir(new_case_path)
        
        try:
            # Operating condition settings
            config_dict = {
                "mechanism": f"{DFODE_ROOT}/mechanisms/Burke2012_s9r23.yaml",
                "T0": 500,
                "p0": 101325,
                "fuel": "H2:1",
                "oxidizer": "O2:0.21,N2:0.79",
                "eq_ratio": eq_ratio,  
            }
            config = OneDFreelyPropagatingFlameConfig(**config_dict)

            # Simulation settings
            settings = {
                "sim_time_step": 1e-6,
                "sim_write_interval": 1e-5,
                "num_output_steps": num_output_steps,
            }
            config.update_config(settings)

            # Setup the case and update dictionary files
            setup_one_d_flame_case(config, '.')
            
            print(f"successfully setting {new_case_name}")
            
        except Exception as e:
            print(f"Wrong in setting {new_case_name}: {e}")
        finally:
            os.chdir(original_dir)
    else:
        print(f"Error")



Note that at the point, the simulation is not yet started. The user would need to ensure a working version of DeepFlame is available and run the `Allrun` script from command line to start the simulation.

```bash
./Allrun
```

After the simulation is completed, we proceed to use DFODE-kit to gather and manage the thermochemical data.

In [ ]:
for eq_ratio in eq_ratios:
    df_to_h5(
        root_dir=f"{DFODE_ROOT}/tutorials/twoD_HIT_flame/1_sample_train/eq_ratio_{eq_ratio}",
        mechanism=f"{DFODE_ROOT}/mechanisms/Burke2012_s9r23.yaml",
        hdf5_file_path=f"{DFODE_ROOT}/tutorials/twoD_HIT_flame/1_sample_train/tutorial_data_{eq_ratio}.h5",
        include_mesh=True,
    )


#### Data augmentation and labeling

While laminar canonical flames provide fundamental thermochemical states,their trajectory-aligned sampling in composition space poses significant limitations for a posteriori modeling applications. First, these sampled states are confined to predefined flamelet manifolds, making the trained model highly sensitive to perturbations and leading to an over-constrained representation. Second, the sampled states span a lower-dimensional subspace, which fails to encompass the full range of thermochemical variations encountered in turbulent combustion. As a result, the model becomes vulnerable to off-manifold perturbations—deviations from the training manifold that frequently arise in turbulent reacting flows.

To tackle this challenge, a data augmentation strategy is employed, where collected states are perturbed to simulate the effects of multi-dimensional transport and turbulence disturbances.

Note:
For this demonstration, a sampling size of 20,000 is adopted per case. This parameter can be adjusted upward in actual implementations to enhance statistical reliability.

In [ ]:
dataset_num = 600000

for eq_ratio in eq_ratios:
    h5_file = f'tutorial_data_{eq_ratio}.h5'
    mech = f'{DFODE_ROOT}/mechanisms/Burke2012_s9r23.yaml'
    output_file = f'data_{eq_ratio}'

    print(f"Loading data from h5 file: {h5_file}")
    data = get_TPY_from_h5(h5_file)    
    print("Data shape:", data.shape)
    All_data = random_perturb(data, mech, dataset_num, heat_limit=False, element_limit=True, eq_ratio=eq_ratio)
    np.save(output_file, All_data)
    print("Saved augmented data shape:", All_data.shape)
    print(f"Saved augmented data to {output_file}")


The CVODE integrator from Cantera is used for time integration and to provide supervised learning labels.

In [ ]:
from dfode_kit.data_operations import label_npy

all_labeled_data = []

for eq_ratio in eq_ratios:
    try:
        labeled_data = label_npy(
            mech_path=mech,
            time_step=1e-06,
            source_path=f'./data_{eq_ratio}.npy'
        )
        all_labeled_data.append(labeled_data)
        
    except (FileNotFoundError, ValueError) as e:
        print(f"Error: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

all_labeled_data = np.vstack(all_labeled_data)
np.save('dataset.npy', all_labeled_data)
print(f"Labeled data saved to: {'dataset.npy'}, shape: {all_labeled_data.shape}")


#### Model training

Only a demo for training a model is provided here.

Note:
The current implementation employs a standard neural network configuration as a foundational reference. Users are encouraged to adapt the network parameters based on their specific problem domains and performance criteria, including the max_poch, the structure of networks, etc.

In [ ]:
import torch
import cantera as ct
from dfode_kit.dfode_core.train.formation import formation_calculate

source_file = 'dataset.npy'
time_step = 1e-06
output_path = './model.pt'

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
labeled_data = np.load(source_file)

gas = ct.Solution(mech)
n_species = gas.n_species
formation_enthalpies = formation_calculate(mech)

# Model instantiation
demo_model = MLP([2+n_species, 400, 400, 400, 400, n_species-1]).to(device)

# Data loading
thermochem_states1 = labeled_data[:, 0:2+n_species]
thermochem_states2 = labeled_data[:, 2+n_species:]

print(thermochem_states1.shape, thermochem_states2.shape)
thermochem_states1[:, 2:] = np.clip(thermochem_states1[:, 2:], 0, 1)
thermochem_states2[:, 2:] = np.clip(thermochem_states2[:, 2:], 0, 1)

features = torch.tensor(np.hstack((thermochem_states1[:, :2], BCT(thermochem_states1[:, 2:]))), dtype=torch.float32).to(device)
labels = torch.tensor(BCT(thermochem_states2[:, 2:-1]) - BCT(thermochem_states1[:, 2:-1]), dtype=torch.float32).to(device)

features_mean = torch.mean(features, dim=0)
features_std = torch.std(features, dim=0)
features = (features - features_mean) / features_std

labels_mean = torch.mean(labels, dim=0)
labels_std = torch.std(labels, dim=0)
labels = (labels - labels_mean) / labels_std

formation_enthalpies = torch.tensor(formation_enthalpies, dtype=torch.float32).to(device)

# Training
loss_fn = torch.nn.L1Loss()

demo_model.train()  

max_epochs = 1500
initial_lr = 0.001
lr_decay_epoch = 500
batch_size = 2000
optimizer = torch.optim.Adam(demo_model.parameters(), lr=initial_lr)


for epoch in range(max_epochs):
    if epoch > 0 and epoch % lr_decay_epoch == 0:
        for param_group in optimizer.param_groups:
            param_group['lr'] *= 0.1
    
    # 初始化损失值
    total_loss1 = 0
    total_loss2 = 0
    total_loss3 = 0
    total_batches = 0
    total_loss = 0

    for i in range(0, len(features), batch_size):
        batch_features = features[i:i + batch_size]
        batch_labels = labels[i:i + batch_size]

        optimizer.zero_grad()

        preds = demo_model(batch_features)
        loss1 = loss_fn(preds, batch_labels)  

        Y_in = ((batch_features[:, 2:-1] * features_std[2:-1] + features_mean[2:-1]) * 0.1 + 1) ** 10
        Y_out = (((preds * labels_std + labels_mean) + (batch_features[:, 2:-1] * features_std[2:-1] + features_mean[2:-1])) * 0.1 + 1) ** 10
        Y_target = (((batch_labels * labels_std + labels_mean) + (batch_features[:, 2:-1] * features_std[2:-1] + features_mean[2:-1])) * 0.1 + 1) ** 10

        loss2 = loss_fn(Y_out.sum(axis=1), Y_in.sum(axis=1))

        Y_out_total = torch.cat((Y_out, (1 - Y_out.sum(axis=1)).reshape(Y_out.shape[0], 1)), axis=1)
        Y_target_total = torch.cat((Y_target, (1 - Y_target.sum(axis=1)).reshape(Y_target.shape[0], 1)), axis=1)

        loss3 = loss_fn((formation_enthalpies * Y_out_total).sum(axis=1), (formation_enthalpies * Y_target_total).sum(axis=1)) / time_step
        loss = loss1 + loss2 + loss3 / 1e+13

        total_loss1 += loss1.item()
        total_loss2 += loss2.item()
        total_loss3 += loss3.item()
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
    
    total_loss1 /= (len(features) / batch_size)
    total_loss2 /= (len(features) / batch_size)
    total_loss3 /= (len(features) / batch_size)
    total_loss /= (len(features) / batch_size)

    print("Epoch: {}, Loss1: {:4e}, Loss2: {:4e}, Loss3: {:4e}, Loss: {:4e}".format(epoch+1, total_loss1, total_loss2, total_loss3, total_loss))

torch.save(
    {
        'net': demo_model.state_dict(),
        'data_in_mean': features_mean.cpu().numpy(),
        'data_in_std': features_std.cpu().numpy(),
        'data_target_mean': labels_mean.cpu().numpy(),
        'data_target_std': labels_std.cpu().numpy(),
    },
    output_path
)

# The above is equivalent to the following cli command:
# dfode-kit train --mech ../../mechanisms/Burke2012_s9r23.yaml     \
#     --source_file ./dataset.npy     \
#     --output_path ./demo_model.pt